<h1> 2. Creating a sampled dataset </h1>

This notebook illustrates:
<ol>
<li> Sampling a BigQuery dataset to create datasets for ML
<li> Preprocessing with Pandas
</ol>

In [1]:
# change these to try this notebook out
BUCKET = 'cloud-training-demos-ml'
PROJECT = 'cloud-training-demos'
REGION = 'us-central1'

In [2]:
import os
os.environ['BUCKET'] = BUCKET
os.environ['PROJECT'] = PROJECT
os.environ['REGION'] = REGION

In [3]:
%%bash
if ! gsutil ls | grep -q gs://${BUCKET}/; then
  gsutil mb -l ${REGION} gs://${BUCKET}
fi

Creating gs://cloud-training-demos-ml/...
ServiceException: 409 Bucket cloud-training-demos-ml already exists.


CalledProcessError: Command 'b'if ! gsutil ls | grep -q gs://${BUCKET}/; then\n  gsutil mb -l ${REGION} gs://${BUCKET}\nfi\n'' returned non-zero exit status 1.

<h2> Create ML dataset by sampling using BigQuery </h2>
<p>
Let's sample the BigQuery data to create smaller datasets.
</p>

In [4]:
# Create SQL query using natality data after the year 2000
from google.cloud import bigquery
query = """
SELECT
  weight_pounds,
  is_male,
  mother_age,
  plurality,
  gestation_weeks,
  FARM_FINGERPRINT(CONCAT(CAST(YEAR AS STRING), CAST(month AS STRING))) AS hashmonth
FROM
  publicdata.samples.natality
WHERE year > 2000
"""

There are only a limited number of years and months in the dataset. Let's see what the hashmonths are.

In [5]:
# Call BigQuery but GROUP BY the hashmonth and see number of records for each group to enable us to get the correct train and evaluation percentages
df = bigquery.Client().query("SELECT hashmonth, COUNT(weight_pounds) AS num_babies FROM (" + query + ") GROUP BY hashmonth").to_dataframe()
print("There are {} unique hashmonths.".format(len(df)))
df.head()

There are 96 unique hashmonths.


,hashmonth,num_babies
0,-1403073183891835564,351299
1,3545707052733304728,327823
2,1451354159195218418,334485
3,-9183605629983195042,329975
4,-2700219941840496344,307395


Here's a way to get a well distributed portion of the data in such a way that the test and train sets do not overlap:

In [6]:
# Added the RAND() so that we can now subsample from each of the hashmonths to get approximately the record counts we want
trainQuery = "SELECT * FROM (" + query + ") WHERE ABS(MOD(hashmonth, 4)) < 3 AND RAND() < 0.0005"
evalQuery = "SELECT * FROM (" + query + ") WHERE ABS(MOD(hashmonth, 4)) = 3 AND RAND() < 0.0005"
traindf = bigquery.Client().query(trainQuery).to_dataframe()
evaldf = bigquery.Client().query(evalQuery).to_dataframe()
print("There are {} examples in the train dataset and {} in the eval dataset".format(len(traindf), len(evaldf)))

There are 13450 examples in the train dataset and 3228 in the eval dataset


<h2> Preprocess data using Pandas </h2>
<p>
Let's add extra rows to simulate the lack of ultrasound. In the process, we'll also change the plurality column to be a string.

In [7]:
traindf.head()

,weight_pounds,is_male,mother_age,plurality,gestation_weeks,hashmonth
0,7.874912,True,32,1,38.0,3095933535584005890
1,9.625382,False,21,1,40.0,3095933535584005890
2,7.749249,True,27,1,42.0,3095933535584005890
3,8.549527,False,30,1,39.0,3095933535584005890
4,6.309630,False,20,1,39.0,3095933535584005890


Also notice that there are some very important numeric fields that are missing in some rows (the count in Pandas doesn't count missing data)

In [8]:
# Let's look at a small sample of the training data
traindf.describe()

,weight_pounds,mother_age,plurality,gestation_weeks,hashmonth
count,13437.000000,13450.000000,13450.000000,13351.000000,1.345000e+04
mean,7.247205,27.328401,1.032491,38.638679,3.577250e+17
std,1.312633,6.212405,0.187497,2.551507,5.238067e+18
min,0.500449,13.000000,1.000000,17.000000,-9.183606e+18
25%,6.563162,22.000000,1.000000,38.000000,-3.340563e+18
50%,7.312733,27.000000,1.000000,39.000000,-2.605984e+17
75%,8.062305,32.000000,1.000000,40.000000,5.896568e+18
max,15.432358,53.000000,5.000000,47.000000,8.599690e+18


In [9]:
# It is always crucial to clean raw data before using in ML, so we have a preprocessing step
import pandas as pd
def preprocess(df):
  # clean up data we don't want to train on
  # in other words, users will have to tell us the mother's age
  # otherwise, our ML service won't work.
  # these were chosen because they are such good predictors
  # and because these are easy enough to collect
  df = df[df.weight_pounds > 0]
  df = df[df.mother_age > 0]
  df = df[df.gestation_weeks > 0]
  df = df[df.plurality > 0]
  
  # modify plurality field to be a string
  twins_etc = dict(zip([1,2,3,4,5],
                   ['Single(1)', 'Twins(2)', 'Triplets(3)', 'Quadruplets(4)', 'Quintuplets(5)']))
  df['plurality'].replace(twins_etc, inplace=True)
  
  # now create extra rows to simulate lack of ultrasound
  nous = df.copy(deep=True)
  nous.loc[nous['plurality'] != 'Single(1)', 'plurality'] = 'Multiple(2+)'
  nous['is_male'] = 'Unknown'
  
  return pd.concat([df, nous])

In [10]:
traindf.head()# Let's see a small sample of the training data now after our preprocessing
traindf = preprocess(traindf)
evaldf = preprocess(evaldf)
traindf.head()

,weight_pounds,is_male,mother_age,plurality,gestation_weeks,hashmonth
0,7.874912,True,32,Single(1),38.0,3095933535584005890
1,9.625382,False,21,Single(1),40.0,3095933535584005890
2,7.749249,True,27,Single(1),42.0,3095933535584005890
3,8.549527,False,30,Single(1),39.0,3095933535584005890
4,6.309630,False,20,Single(1),39.0,3095933535584005890


In [ ]:
traindf.tail()

In [ ]:
# Describe only does numeric columns, so you won't see plurality
traindf.describe()

<h2> Write out </h2>
<p>
In the final versions, we want to read from files, not Pandas dataframes. So, write the Pandas dataframes out as CSV files. 
Using CSV files gives us the advantage of shuffling during read. This is important for distributed training because some workers might be slower than others, and shuffling the data helps prevent the same data from being assigned to the slow workers.


In [ ]:
traindf.to_csv('train.csv', index=False, header=False)
evaldf.to_csv('eval.csv', index=False, header=False)

In [ ]:
%%bash
wc -l *.csv
head *.csv
tail *.csv

Copyright 2017-2018 Google Inc. Licensed under the Apache License, Version 2.0 (the "License"); you may not use this file except in compliance with the License. You may obtain a copy of the License at http://www.apache.org/licenses/LICENSE-2.0 Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License